In [15]:
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv 

import csv
import json

Extraer detalles como:

Artista -> de mis artistas  de spotify -> traer CSV??? 🚨🚨

País de Origen.

Area de Origen.

Fechas inicio de actividad.

Fechas fin de actividad (en caso que tenga o simplemente "actualidad").

Llamada con Endpoint en funcion de los artistas de spotify

Ejemplo de endpoint: https://musicbrainz.org/ws/2/release/?query=album:Abbey%20Road&fmt=json

In [2]:
# Llamada a la API

url = "https://musicbrainz.org/ws/2/artist/?query=artist:shakira&fmt=json"


# Hemos sacado key pero de momento no nos hace falta para la llamada
# api_key = "RjTkSzMRh0RiiMO9oyJicZYIuNPLYVkhVbSRtjDB"
# headers = {"Authorization": api_key}



In [3]:
def llamar_api(url):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.
    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.

In [4]:
result = llamar_api(url)
result

La llamada a la API nos ha dado una respuesta de tipo: 200


{'created': '2024-10-23T10:20:48.890Z',
 'count': 13,
 'offset': 0,
 'artists': [{'id': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
   'type': 'Person',
   'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df',
   'score': 100,
   'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2',
   'name': 'Shakira',
   'sort-name': 'Shakira',
   'gender': 'female',
   'country': 'CO',
   'area': {'id': '02b60d8d-7164-339d-868d-22d147d9f74a',
    'type': 'Country',
    'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
    'name': 'Colombia',
    'sort-name': 'Colombia',
    'life-span': {'ended': None}},
   'begin-area': {'id': 'c75b683f-3238-4fc8-b5ca-1883ab911561',
    'type': 'City',
    'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
    'name': 'Barranquilla',
    'sort-name': 'Barranquilla',
    'life-span': {'ended': None}},
   'disambiguation': 'Colombian singer',
   'ipis': ['00353347465', '00353347661'],
   'isnis': ['0000000116252147', '0000000132386436'],
   'life-span': {'begin': '1977-02-

In [5]:
result.keys()

dict_keys(['created', 'count', 'offset', 'artists'])

In [6]:
result["count"]

13

In [7]:
# Podemos dar por supuesto que es el primer elemento que nos de o iterar entre los dict que tiene la lista resultado y que nos de el valor mas exacto en el name
result["artists"][0]["type"]

'Person'

In [8]:
# Pais origen
result["artists"][0]["area"]["name"]



'Colombia'

In [9]:
# Area origen
result["artists"][0]["begin-area"]["name"]


'Barranquilla'

In [10]:
# Fecha inicio de actividad (si lo hacemos como grupo vs)

result["artists"][0]["life-span"]["begin"]

'1977-02-02'

In [11]:
# Fecha fin de actividad

result["artists"][0]["life-span"]["ended"]

In [ ]:


for i in result["artists"][0]: 
    print(i)

In [17]:
# Paso 1: Leer el archivo CSV
spoti_csv_file = "../data/dataframes_combinados.csv" # nombre de tu archivo Spoti CSV
spoti_json_file = "dataframes_combinados.json"  # nombre del archivo Spoti de salida JSON

# Paso 2: Abrir el CSV y convertir a formato JSON
with open(spoti_csv_file, mode='r', encoding='utf-8') as file:
    # Leer el contenido CSV
    csv_reader = csv.DictReader(file)
    
    # Convertir cada fila a diccionario y agregarla a una lista
    data = [row for row in csv_reader]

# Paso 3: Escribir los datos en un archivo JSON
with open(spoti_json_file, mode='w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)  # Guarda el JSON con formato legible (indent=4)

print(f'Archivo {spoti_json_file} creado con éxito.')

Archivo dataframes_combinados.json creado con éxito.


In [20]:
#Traemos el archivo a json
with open("dataframes_combinados.json","r") as f: 
    result_spoti = json.load(f)

In [25]:
#Creamos lista vacia para que se complete con los artistas
artist = []

for item in result_spoti: 
    artist.append(item['artista'])

In [29]:
#Convertimos en set para eliminar artistas duplicados (hay artistas con mas de una canción en el mismo año)
set_artist = set(artist)

In [ ]:

url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{set_artist}&fmt=json"

In [ ]:
dict_mb = {"artista" : [], "pais_de_origen" : [], "area_de_origen" : [], "fecha_nacimiento" : [], "fecha_inicio_actividad" : [], "fecha_fin_actividad" :[]}

for artista in set_artist: 
    

In [33]:
result

{'created': '2024-10-23T10:20:48.890Z',
 'count': 13,
 'offset': 0,
 'artists': [{'id': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
   'type': 'Person',
   'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df',
   'score': 100,
   'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2',
   'name': 'Shakira',
   'sort-name': 'Shakira',
   'gender': 'female',
   'country': 'CO',
   'area': {'id': '02b60d8d-7164-339d-868d-22d147d9f74a',
    'type': 'Country',
    'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
    'name': 'Colombia',
    'sort-name': 'Colombia',
    'life-span': {'ended': None}},
   'begin-area': {'id': 'c75b683f-3238-4fc8-b5ca-1883ab911561',
    'type': 'City',
    'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
    'name': 'Barranquilla',
    'sort-name': 'Barranquilla',
    'life-span': {'ended': None}},
   'disambiguation': 'Colombian singer',
   'ipis': ['00353347465', '00353347661'],
   'isnis': ['0000000116252147', '0000000132386436'],
   'life-span': {'begin': '1977-02-

In [56]:
result["artists"][0]

{'id': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
 'type': 'Person',
 'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df',
 'score': 100,
 'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2',
 'name': 'Shakira',
 'sort-name': 'Shakira',
 'gender': 'female',
 'country': 'CO',
 'area': {'id': '02b60d8d-7164-339d-868d-22d147d9f74a',
  'type': 'Country',
  'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
  'name': 'Colombia',
  'sort-name': 'Colombia',
  'life-span': {'ended': None}},
 'begin-area': {'id': 'c75b683f-3238-4fc8-b5ca-1883ab911561',
  'type': 'City',
  'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
  'name': 'Barranquilla',
  'sort-name': 'Barranquilla',
  'life-span': {'ended': None}},
 'disambiguation': 'Colombian singer',
 'ipis': ['00353347465', '00353347661'],
 'isnis': ['0000000116252147', '0000000132386436'],
 'life-span': {'begin': '1977-02-02', 'ended': None},
 'aliases': [{'sort-name': 'Xiaqila',
   'type-id': '894afba6-2816-3c24-8072-eadb66bd04bc',
   'name': '夏奇拉'

In [70]:
dic_shakira = {"artista" : [], "pais_de_origen" : [], "area_de_origen" : []}

for i in result["artists"]: 
   print(i)
   print(result["artists"])
   print(i["name"])
   print(i["area"]["name"])
   print(i["begin-area"]["name"])
   break
   

{'id': 'bf24ca37-25f4-4e34-9aec-460b94364cfc', 'type': 'Person', 'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df', 'score': 100, 'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2', 'name': 'Shakira', 'sort-name': 'Shakira', 'gender': 'female', 'country': 'CO', 'area': {'id': '02b60d8d-7164-339d-868d-22d147d9f74a', 'type': 'Country', 'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de', 'name': 'Colombia', 'sort-name': 'Colombia', 'life-span': {'ended': None}}, 'begin-area': {'id': 'c75b683f-3238-4fc8-b5ca-1883ab911561', 'type': 'City', 'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78', 'name': 'Barranquilla', 'sort-name': 'Barranquilla', 'life-span': {'ended': None}}, 'disambiguation': 'Colombian singer', 'ipis': ['00353347465', '00353347661'], 'isnis': ['0000000116252147', '0000000132386436'], 'life-span': {'begin': '1977-02-02', 'ended': None}, 'aliases': [{'sort-name': 'Xiaqila', 'type-id': '894afba6-2816-3c24-8072-eadb66bd04bc', 'name': '夏奇拉', 'locale': 'zh', 'type': 'Artist name', 

In [58]:
dic_shakira

{'artista': [], 'pais_de_origen': [], 'area_de_origen': []}

In [71]:
dic_shakira = {"artista" : [], "pais_de_origen" : [], "area_de_origen" : [], "fecha_nacimiento" : [], "fecha_inicio_actividad" : [], "fecha_fin_actividad" :[]}

for i in result["artists"]: 
    
   dic_shakira["artista"].append(i["name"])
   dic_shakira["pais_de_origen"].append(i["area"]["name"])  

KeyError: 'area'